In [1]:
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras import optimizers
from keras.models import Sequential, Model 
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D
from keras import backend as k 
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping
from keras.applications.resnet50 import ResNet50
from keras.models import load_model
import tensorflow as tf
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt
import pandas as pd
from keras.utils import plot_model

Using TensorFlow backend.


In [2]:
train_path = 'X:/Flipkart GRID - Object Localization (Medium Complexity Round)/augument_data/training_images'

img_width, img_height = 640., 480.

In [3]:
##### img_width, img_height = 640, 480
base_dir = 'X:/Flipkart GRID - Object Localization (Medium Complexity Round)/'
# train_data_dir = "X:/train-scene classification/data/train/"
# validation_data_dir = "X:/train-scene classification/data/valid/"
# test_data_dir = "X:/train-scene classification/data/test/"
epochs = 1000
# Save the model according to the conditions  
chkpt_dir = "X:/Flipkart GRID - Object Localization (Medium Complexity Round)/chkpts/resnet50model-2.h5" # Change for every new training session 

In [4]:
# base_model = ResNet50(weights = "imagenet", include_top=False, input_shape = (img_height, img_width, 3), classes = 4)

In [5]:
# base_model.save('resnet50.h5')
base_model = load_model(os.path.join(base_dir,'base_models', 'resnet50.h5'))

C:\Users\praty\Anaconda3\envs\dlenv\lib\site-packages\keras\engine\saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [6]:
for layer in base_model.layers[:-5]:
    layer.trainable = False
base_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 480, 640, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 486, 646, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 240, 320, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 240, 320, 64) 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

In [36]:
# for layer in base_model.layers:
#     layer.trainable = False
x = base_model.output
x = Flatten()(x)
# x = Dense(2048, activation="relu")(x)
# x = Dropout(0.7)(x)
x = Dense(16, activation="relu")(x)
x = Dropout(0.05)(x)
# x = Dense(64, activation="relu")(x)
# x = Dropout(0.0)(x)
# x = Dense(1024, activation="relu")(x)
# x = Dropout(0.5)(x)
# x = Dense(512, activation="relu")(x)
# x = Dropout(0.3)(x)
# predictions = Dense(6, activation="softmax")(x)
predictions = Dense(4, activation="linear")(x)

In [37]:
model = Model(input = base_model.input, output = predictions)
# for layer in model.layers[:-3]:
#     layer.trainable = False
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 480, 640, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 486, 646, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 240, 320, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 240, 320, 64) 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

C:\Users\praty\Anaconda3\envs\dlenv\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  """Entry point for launching an IPython kernel.


In [38]:
def mIoU(labels,predictions):
#     print(labels.shape)
#     print(predictions.shape)
#     print(labels[0])
#     print(predictions[0])
    pred_x1=predictions[:,0]#*img_width
    pred_x2=predictions[:,1]#*img_width
    pred_y1=predictions[:,2]#*img_height
    pred_y2=predictions[:,3]#*img_height
    label_x1=labels[:,0]#*img_width
    label_x2=labels[:,1]#*img_width
    label_y1=labels[:,2]#*img_height
    label_y2=labels[:,3]#*img_height
    int_x1 = tf.maximum(pred_x1, label_x1)
    int_x2 = tf.minimum(pred_x2, label_x2)
    int_y1 = tf.maximum(pred_y1, label_y1)
    int_y2 = tf.minimum(pred_y2, label_y2)
    tensor_type = pred_x1.dtype
    pred_area = tf.multiply(tf.maximum(tf.cast(0.0, tensor_type), pred_x2 - pred_x1), tf.maximum(tf.cast(0.0, tensor_type), pred_y2 - pred_y1))
    label_area = tf.multiply(tf.maximum(tf.cast(0.0, tensor_type), label_x2 - label_x1), tf.maximum(tf.cast(0.0, tensor_type), label_y2 - label_y1))
    int_area = tf.multiply(tf.maximum(tf.cast(0.0, tensor_type), int_x2 - int_x1), tf.maximum(tf.cast(0.0, tensor_type), int_y2 - int_y1))
    union_area = pred_area + label_area - int_area
    IoU = int_area / union_area
    return tf.reduce_mean(IoU)

In [39]:
# IoU Metric Test
LABEL = [[0.0, 1.0, 0.0, 1.0], 
        [2.0, 3.0, 2.0, 3.0]]
PRED = [[0.5, 1.5, 0.5, 1.5],
        [2.0, 3.0, 2.0, 3.0]]
LABEL = tf.constant(np.array(LABEL))
PRED = tf.constant(np.array(PRED))
print(mIoU(LABEL, PRED))
with tf.Session() as sess:
    val = sess.run(mIoU(LABEL, PRED))
    print(val)

Tensor("Mean_6:0", shape=(), dtype=float64)
0.5714285714285714


In [40]:
model.load_weights(chkpt_dir)

In [41]:
model.compile(loss = "mse", optimizer = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-8), metrics=['mse', mIoU])

In [42]:
checkpoint = ModelCheckpoint(chkpt_dir, monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
early = EarlyStopping(monitor='val_loss', min_delta=0, patience=40, verbose=1, mode='auto')

In [43]:
train_df = pd.read_csv(os.path.join(base_dir, 'augument_data', 'augmented_train_data.csv'))
val_df = pd.read_csv(os.path.join(os.path.join(base_dir, 'augument_data', 'augmented_val_data.csv')))

In [15]:
l = []
for im in val_df.image_name:
    img = cv2.imread(os.path.join(train_path, im))
    l.append(np.expand_dims(img, axis = 0))
x_val = np.vstack(l)
y_val = val_df.iloc[:,1:].values.astype(np.float64)

y_val[:, 0] = y_val[:, 0]# / img_width
y_val[:, 1] = y_val[:, 1]# / img_width
y_val[:, 2] = y_val[:, 2]# / img_height
y_val[:, 3] = y_val[:, 3]# / img_height
print(y_val[0])
print(y_val.dtype)
print(x_val.shape)

[156. 546. 146. 382.]
float64
(4000, 480, 640, 3)


In [16]:
# model.load_weights(chkpt_dir)

In [44]:
for epoch in range(epochs):
    print(epoch)
    print("--------------------------------------------------------------")
    for i in range(66):
        print('batch {}/66 (epoch {}):'.format(i, epoch))
        l = []
        for im in train_df.image_name[i * 1000: (i + 1) * 1000]:
            img = cv2.imread(os.path.join(train_path, im))
            l.append(np.expand_dims(img, axis = 0))
        x_train = np.vstack(l) # IMAGES NOT NORMALIZED!!!
        y_train = train_df[i * 1000: (i + 1) * 1000].iloc[:,1:].values
        y_train[:, 0] = y_train[:, 0]# / img_width
        y_train[:, 1] = y_train[:, 1]# / img_width
        y_train[:, 2] = y_train[:, 2]# / img_height
        y_train[:, 3] = y_train[:, 3]# / img_height
        
        model.fit(x=x_train, y=y_train, batch_size=32, epochs=1, verbose=1, callbacks=[checkpoint], validation_split=0.0, validation_data=(x_val, y_val), shuffle=True)

0
--------------------------------------------------------------
batch 0/66 (epoch 0):
Train on 1000 samples, validate on 4000 samples
Epoch 1/1
1000/1000 [==============================] - 101s 101ms/step - loss: 5569.2954 - mean_squared_error: 5569.2954 - mIoU: 0.5781 - val_loss: 4084.8768 - val_mean_squared_error: 4084.8768 - val_mIoU: 0.5277

Epoch 00001: val_loss improved from inf to 4084.87675, saving model to X:/Flipkart GRID - Object Localization (Medium Complexity Round)/chkpts/resnet50model-2.h5
batch 1/66 (epoch 0):
Train on 1000 samples, validate on 4000 samples
Epoch 1/1
1000/1000 [==============================] - 100s 100ms/step - loss: 4815.8644 - mean_squared_error: 4815.8644 - mIoU: 0.6113 - val_loss: 2672.8032 - val_mean_squared_error: 2672.8032 - val_mIoU: 0.6198

Epoch 00001: val_loss improved from 4084.87675 to 2672.80317, saving model to X:/Flipkart GRID - Object Localization (Medium Complexity Round)/chkpts/resnet50model-2.h5
batch 2/66 (epoch 0):
Train on 1000 

KeyboardInterrupt: 

In [ ]:
model.save('transfer_resnet50-2.h5')

In [ ]:
model.load_weights(chkpt_dir)
LABEL = y_val
PRED = model.predict(x_val)
with tf.Session() as sess:
    val = sess.run(mIoU(LABEL, PRED))
    print(val)